In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import urllib.request
import os
import zipfile
import tarfile
import pickle
import shutil
# don't move this cell

In [ ]:
# If you open the notebook on Colab, pull the all code files via git clone
# comment out if you are on a local machine
!git clone https://github.com/Jacopo-DM/FACT-2022.git
!mv  -v FACT-2022/* /content/
!rm -r FACT-2022

In [ ]:
!pip install -r requirements.txt

#### Download datasets and model files

CUB-200 Dataset

In [ ]:
if not os.path.exists('data'):
    os.mkdir('data')

if not os.path.exists('data/CUB200'):
    print('Beginning file download with urllib2...')
    url = 'https://data.deepai.org/CUB200(2011).zip'
    urllib.request.urlretrieve(url, 'data/CUB200.zip')

    with zipfile.ZipFile('data/CUB200.zip', 'r') as zip_ref:
        zip_ref.extractall('data/CUB200')

    my_tar = tarfile.open('data/CUB200/CUB_200_2011.tgz')
    my_tar.extractall('data/CUB200/') # specify which folder to extract to
    my_tar.close()

    os.remove('data/CUB200.zip')
    os.remove('data/CUB200/CUB_200_2011.tgz')

Downloading the trained model files and the ImageNet ILSVRC 2012-2017 Challenge validation set. 

For the full (not needed for this result notebook) ImageNet ILSVRC dataset including training data, see https://www.kaggle.com/c/imagenet-object-localization-challenge/data.

#### IMPORTANT NOTE

All files can be found in this public drive: https://drive.google.com/drive/folders/1RpmzTNrc8i1eSu7C-Mctdhy2SZiXbH80?usp=sharing

Please use this Google Drive link and copy the files to the correct directories so they can be unzipped by the code. The zipfiles should be in the following directories:
```
imgnet_val.zip in data/
val_annotations.zip in data/
model_files_archive.zip in /    (the main directory of the project)

```

The ```data/``` directory is only created when the previous cell for downloading CUB-200 is done running.

#### In case you use Google Colab (fastest method)
In case of Colab you can mount your Google Drive. You should first make a copy of all files in the shared link to your own Google Drive. Eventually, you can access these files in Colab after you have mounted Google Drive, see the code below.


In [ ]:
## Only use this when in colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Only use this when using colab with mounted google drive and you copied the zip files to you own google drive
# Change to correct path after Google Drive was mounted
# This code moves the zip files into the right directories for extracting
!rsync -ah --progress drive/MyDrive/imgnet_val.zip data/
!rsync -ah --progress drive/MyDrive/val_annotations.zip data/
!rsync -ah --progress drive/MyDrive/model_files_archive.zip /content/

In [ ]:
#### UNZIP CODE ####

# unzip imagenet images
if not os.path.exists('data/imagenet'):
    with zipfile.ZipFile('data/imgnet_val.zip', 'r') as zip_ref:
        zip_ref.extractall('data/imagenet/')

if os.path.exists('data/imgnet_val.zip'):
    os.remove('data/imgnet_val.zip')


# unzip annotations
with zipfile.ZipFile('data/val_annotations.zip', 'r') as zip_ref:
    zip_ref.extractall('data/imagenet/annotations')

if os.path.exists('data/val_annotations.zip'):
    os.remove('data/val_annotations.zip')

# create empty train data folder to avoid downloading 
# the full size dataset incl. training.
with open('categories.pickle', 'rb') as f:
    cats = pickle.load(f)
if not os.path.exists('data/imagenet/train'):
    os.mkdir('data/imagenet/train')
    for cat in cats:
        os.mkdir('data/imagenet/train/{}'.format(cat))

# download trained model files
if not os.path.exists('trained_model_files'):
    os.mkdir('trained_model_files')
with zipfile.ZipFile('model_files_archive.zip', 'r') as zip_ref:
    zip_ref.extractall('trained_model_files/')

shutil.rmtree('trained_model_files/__MACOSX')

if os.path.exists('model_files_archive.zip'):
    os.remove('model_files_archive.zip')

#####################################

## Calculate metrics and visualize results

Please make sure you are running this on a GPU (we used Google Colab).

If you run into an error like this:

```RuntimeError: unexpected EOF, expected 5253807 more bytes. The file might be corrupted```

something went wrong with unzipping or copying the model files which made them corrupt. In this case you should download all model files via this link:

https://www.dropbox.com/sh/fo9kqm0fgjhoq5j/AAA-A2cznqSsZuP31iLvqWKta?dl=0

When the files were downloaded, please store them in a directory called ```trained_model_files```. The directory ```trained_model_files``` should be stored in the main directory of the project.

####Parameter setup

Parameters for ImageNet with positive SCOUTER for different lambda values




In [ ]:
params_imgnet_n_100_lambda1 = ['--dataset', 'ImageNet', '--model', 'resnest26d', '--batch_size','70', '--epochs','20', '--num_classes','100',
'--use_slot','true', '--use_pre', 'false', '--loss_status','1', '--slots_per_class','1', '--lambda_value','1',
'--vis','true', '--power', '2', '--to_k_layer', '3', '--channel', '2048', '--freeze_layers','0', '--dataset_dir','data/imagenet/', '--output_dir','trained_model_files/ImageNet_use_slot_checkpoint_classes100_lambda1.pth']

params_imgnet_n_100_lambda3 = ['--dataset', 'ImageNet', '--model', 'resnest26d', '--batch_size','70', '--epochs','20', '--num_classes','100',
'--use_slot','true', '--use_pre', 'false', '--loss_status','1', '--slots_per_class','1', '--lambda_value','3',
'--vis','true', '--power', '2', '--to_k_layer', '3', '--channel', '2048', '--freeze_layers','0', '--dataset_dir','data/imagenet/', '--output_dir','trained_model_files/ImageNet_use_slot_checkpoint_classes100_lambda3.pth']

params_imgnet_n_100_lambda10 = ['--dataset', 'ImageNet', '--model', 'resnest26d', '--batch_size','70', '--epochs','20', '--num_classes','100',
'--use_slot','true', '--use_pre', 'false', '--loss_status','1', '--slots_per_class','1', '--lambda_value','10',
'--vis','true', '--power', '2', '--to_k_layer', '3', '--channel', '2048', '--freeze_layers','0', '--dataset_dir','data/imagenet/', '--output_dir','trained_model_files/ImageNet_use_slot_checkpoint_classes100_lambda10.pth']

param_list_pos = [params_imgnet_n_100_lambda1, params_imgnet_n_100_lambda3, params_imgnet_n_100_lambda10]

Parameters for ImageNet with negative SCOUTER for different lambda values


In [ ]:
params_imgnet_n_100_lambda1_neg = ['--dataset', 'ImageNet', '--model', 'resnest26d', '--batch_size','70', '--epochs','20', '--num_classes','100',
'--use_slot','true', '--use_pre', 'false', '--loss_status','-1', '--slots_per_class','1', '--lambda_value','1',
'--vis','true', '--power', '2', '--to_k_layer', '3', '--channel', '2048', '--freeze_layers','0', '--dataset_dir','data/imagenet/', '--output_dir','trained_model_files/ImageNet_use_slot_negative_checkpoint_classes100_lambda1.pth']

params_imgnet_n_100_lambda3_neg = ['--dataset', 'ImageNet', '--model', 'resnest26d', '--batch_size','70', '--epochs','20', '--num_classes','100',
'--use_slot','true', '--use_pre', 'false', '--loss_status','-1', '--slots_per_class','1', '--lambda_value','3',
'--vis','true', '--power', '2', '--to_k_layer', '3', '--channel', '2048', '--freeze_layers','0', '--dataset_dir','data/imagenet/', '--output_dir','trained_model_files/ImageNet_use_slot_negative_checkpoint_classes100_lambda3.pth']

params_imgnet_n_100_lambda10_neg = ['--dataset', 'ImageNet', '--model', 'resnest26d', '--batch_size','70', '--epochs','20', '--num_classes','100',
'--use_slot','true', '--use_pre', 'false', '--loss_status','-1', '--slots_per_class','1', '--lambda_value','10',
'--vis','true', '--power', '2', '--to_k_layer', '3', '--channel', '2048', '--freeze_layers','0', '--dataset_dir','data/imagenet/', '--output_dir','trained_model_files/ImageNet_use_slot_negative_checkpoint_classes100_lambda10.pth']

param_list_neg = [params_imgnet_n_100_lambda1_neg, params_imgnet_n_100_lambda3_neg, params_imgnet_n_100_lambda10_neg]

CUB-200 positive SCOUTER

In [ ]:
params_cub_n_100_lambda10 = ['--dataset', 'CUB200', '--model', 'resnest26d', '--batch_size','64', '--epochs','150', '--num_classes','100',
'--use_slot','true', '--use_pre', 'true', '--loss_status','1', '--slots_per_class','5', '--lambda_value','10',
'--vis','true', '--power', '2', '--to_k_layer', '3', '--channel', '2048', '--freeze_layers','2', '--dataset_dir','data/CUB200/CUB_200_2011', '--output_dir','trained_model_files/CUB200_use_slot_checkpoint_classes100_lambda10.pth']

param_list_pos_cub = [params_cub_n_100_lambda10]

CUB-200 negative SCOUTER

In [ ]:
params_cub_n_100_lambda10_neg = ['--dataset', 'CUB200', '--model', 'resnest26d', '--batch_size','64', '--epochs','150', '--num_classes','100',
'--use_slot','true', '--use_pre', 'true', '--loss_status','-1', '--slots_per_class','3', '--lambda_value','10',
'--vis','true', '--power', '2', '--to_k_layer', '3', '--channel', '2048', '--freeze_layers','2', '--dataset_dir','data/CUB200/CUB_200_2011', '--output_dir','trained_model_files/CUB200_use_slot_negative_checkpoint_classes100_lambda10.pth']

param_list_neg_cub = [params_cub_n_100_lambda10_neg]

Parameters for the metrics of positive ImageNet SCOUTER
 

In [ ]:
eval_Imagenet100_positive_lambda10 = ['--dataset', 'ImageNet', '--model', 'resnest26d', '--batch_size', '70', '--epochs', '1', '--num_classes', '100', '--use_slot', 'True', '--use_pre', 'false',
'--loss_status', '1', '--slots_per_class', '1', '--power', '2', '--to_k_layer','3', '--lambda_value', '10', '--vis', 'false','--channel', '2048', '--freeze_layers', '0', 
'--dataset_dir', 'data/imagenet/', '--resume', 'true', '--annotations_dir', 'data/imagenet/annotations/', '--pre_dir', 'trained_model_files/ImageNet_use_slot_checkpoint_classes100_lambda10.pth']

eval_Imagenet100_positive_lambda3 =['--dataset', 'ImageNet', '--model', 'resnest26d', '--batch_size', '70', '--epochs', '1', '--num_classes', '100', '--use_slot', 'True', '--use_pre', 'false',
'--loss_status', '1', '--slots_per_class', '1', '--power', '2', '--to_k_layer','3', '--lambda_value', '3', '--vis', 'false','--channel', '2048', '--freeze_layers', '0', 
'--dataset_dir', 'data/imagenet/', '--resume', 'true', '--annotations_dir', 'data/imagenet/annotations/', '--pre_dir', 'trained_model_files/ImageNet_use_slot_checkpoint_classes100_lambda3.pth']

eval_Imagenet100_positive_lambda1 =['--dataset', 'ImageNet', '--model', 'resnest26d', '--batch_size', '70', '--epochs', '1', '--num_classes', '100', '--use_slot', 'True', '--use_pre', 'false',
'--loss_status', '1', '--slots_per_class', '1', '--power', '2', '--to_k_layer','3', '--lambda_value', '1', '--vis', 'false','--channel', '2048', '--freeze_layers', '0', 
'--dataset_dir', 'data/imagenet/', '--resume', 'true', '--annotations_dir', 'data/imagenet/annotations/', '--pre_dir', 'trained_model_files/ImageNet_use_slot_checkpoint_classes100_lambda1.pth']

Parameters for the metrics of negative ImageNet SCOUTER

In [ ]:
eval_Imagenet100_negative_lambda10 = ['--dataset', 'ImageNet', '--model', 'resnest26d', '--batch_size', '70', '--epochs', '1', '--num_classes', '100', '--use_slot', 'True', '--use_pre', 'false',
'--loss_status', '-1', '--slots_per_class', '1', '--power', '2', '--to_k_layer','3', '--lambda_value', '10', '--vis', 'false','--channel', '2048', '--freeze_layers', '0', 
'--dataset_dir', 'data/imagenet/', '--resume', 'true', '--annotations_dir', 'data/imagenet/annotations/', '--pre_dir', 'trained_model_files/ImageNet_use_slot_negative_checkpoint_classes100_lambda10.pth']

eval_Imagenet100_negative_lambda3 =['--dataset', 'ImageNet', '--model', 'resnest26d', '--batch_size', '70', '--epochs', '1', '--num_classes', '100', '--use_slot', 'True', '--use_pre', 'false',
'--loss_status', '-1', '--slots_per_class', '1', '--power', '2', '--to_k_layer','3', '--lambda_value', '3', '--vis', 'false','--channel', '2048', '--freeze_layers', '0', 
'--dataset_dir', 'data/imagenet/', '--resume', 'true', '--annotations_dir', 'data/imagenet/annotations/', '--pre_dir', 'trained_model_files/ImageNet_use_slot_negative_checkpoint_classes100_lambda3.pth']

eval_Imagenet100_negative_lambda1 =['--dataset', 'ImageNet', '--model', 'resnest26d', '--batch_size', '70', '--epochs', '1', '--num_classes', '100', '--use_slot', 'True', '--use_pre', 'false',
'--loss_status', '-1', '--slots_per_class', '1', '--power', '2', '--to_k_layer','3', '--lambda_value', '1', '--vis', 'false','--channel', '2048', '--freeze_layers', '0', 
'--dataset_dir', 'data/imagenet/', '--resume', 'true', '--annotations_dir', 'data/imagenet/annotations/', '--pre_dir', 'trained_model_files/ImageNet_use_slot_negative_checkpoint_classes100_lambda1.pth']

### Visualization of increasing lambda values

#### Positive SCOUTER on ImageNet with 100 classes

In [ ]:
%pylab inline

#
# specifiy datapoint to show results on ImageNet or CUB-200
# NOTE: don't fill in a higher number than the batchsize for the first number in the tuple
# NOTE: also, the second number can't be higher than the batch size
datapoint = (2, 54)

# when batch_size = 70
assert datapoint[1] < 70
from test import *

# create the visualizations
preds = []
target_labels = []
for p in param_list_pos:
    pred, target_label = create_vis(p, datapoint)
    preds.append(pred)
    target_labels.append(target_label)

print(preds, target_labels)

# plot
fig = plt.figure(figsize=(20,40))  
lambda_values = [1, 3, 10]
img = mpimg.imread('sloter/vis/image.png')
plt.subplot(1,4, 1) 
plt.title('Original image')
plt.imshow(img)
for index, l in enumerate(lambda_values):
    img = mpimg.imread('sloter/vis/slot_mask_{}_lambda_{}.png'.format(target_labels[index], l))
    plt.subplot(1,4, index+2) 
    plt.title('$\lambda = {}$'.format(l))
    plt.imshow(img)
    
plt.show()

#### Negative SCOUTER on ImageNet with 100 classes

In [ ]:
%pylab inline

#
# specifiy datapoint to show results on ImageNet or CUB-200
# NOTE: don't fill in a higher number than the number of samples in a batch
# NOTE: also, the second number can't be higher than the batch size
datapoint = (24, 25)

# when batch_size = 70
assert datapoint[1] < 70

from test import *
# create the visualizations
preds = []
target_labels = []
for p in param_list_neg:
    pred, target_label = create_vis(p, datapoint)
    preds.append(pred)
    target_labels.append(target_label)

print(preds, target_labels)

# plot
fig = plt.figure(figsize=(20,40))  
lambda_values = [1, 3, 10]
img = mpimg.imread('sloter/vis/image.png')
plt.subplot(1,4, 1) 
plt.title('Original image')
plt.imshow(img)
for index, l in enumerate(lambda_values):
    img = mpimg.imread('sloter/vis/slot_mask_{}_lambda_{}.png'.format(preds[index], l))
    plt.subplot(1,4, index+2) 
    plt.title('$\lambda = {}$'.format(l))
    plt.imshow(img)
    
plt.show()

## CUB-200 visualization for lambda = 10

#### Positive SCOUTER on CUB-200 with 100 classes

In [ ]:
%pylab inline

#
# specifiy datapoint to show results on ImageNet or CUB-200
# NOTE: don't fill in a higher number than the batchsize for the first number in the tuple
# NOTE: also, the second number can't be higher than the batch size
datapoint = (24, 26)

# when batch_size = 70
assert datapoint[1] < 70

from test import *
# create the visualizations
preds = []
target_labels = []
for p in param_list_pos_cub:
    pred, target_label = create_vis(p, datapoint)
    preds.append(pred)
    target_labels.append(target_label)

print(preds, target_labels)

# plot
fig = plt.figure(figsize=(20,40))  
lambda_values = [10]
img = mpimg.imread('sloter/vis/image.png')
plt.subplot(1,4, 1) 
plt.title('Original image')
plt.imshow(img)
for index, l in enumerate(lambda_values):
    img = mpimg.imread('sloter/vis/slot_mask_{}_lambda_{}.png'.format(target_labels[index], l))
    plt.subplot(1,4, index+2) 
    plt.title('$\lambda = {}$'.format(l))
    plt.imshow(img)
    
plt.show()

#### Positive SCOUTER on CUB-200 with 100 classes

In [ ]:
%pylab inline

# specifiy datapoint to show results on ImageNet or CUB-200
# NOTE: don't fill in a higher number than the batchsize for the first number in the tuple
# NOTE: also, the second number can't be higher than the batch size
datapoint = (24, 26)

# when batch_size = 70
assert datapoint[1] < 70

from test import *
# create the visualizations
preds = []
target_labels = []
for p in param_list_neg_cub:
    pred, target_label = create_vis(p, datapoint)
    preds.append(pred)
    target_labels.append(target_label)

print(preds, target_labels)

# plot
fig = plt.figure(figsize=(20,40))  
lambda_values = [10]
img = mpimg.imread('sloter/vis/image.png')
plt.subplot(1,4, 1) 
plt.title('Original image')
plt.imshow(img)
for index, l in enumerate(lambda_values):
    img = mpimg.imread('sloter/vis/slot_mask_{}_lambda_{}.png'.format(target_labels[index], l))
    plt.subplot(1,4, index+2) 
    plt.title('$\lambda = {}$'.format(l))
    plt.imshow(img)
    
plt.show()

### Training and validation curves

Plotting the training and validation accuracy + loss curves for CUB-200 dataset, trained on different numbers of classes on ResNest50.
We used lambda value 10 for positive SCOUTER and lambda value 1 for negative SCOUTER.

In [ ]:
with open('FC_cub200.pickle', 'rb') as handle:
    FC = pickle.load(handle)

with open('neg_cub200_L1.pickle', 'rb') as handle:
    neg_L1 = pickle.load(handle)

with open('pos_cub200_L10.pickle', 'rb') as handle:
    pos_L10 = pickle.load(handle)

In [ ]:
n_classes = [50, 75, 100]
epochs = 150
  

for i, d in enumerate([FC, neg_L1, pos_L10]):
    fig = plt.figure(figsize=(20,5))
    for i, n in enumerate(n_classes):

        x = range(1, epochs+1)
        plt.subplot(1,3, i+1) 
        y = d[2][n]
        plt.plot(x,y)
        y = d[3][n]
        plt.plot(x,y)
        plt.title(f'Number of classes = {n}')
        plt.legend(['Training accuracy', 'Validation accuracy'])
        plt.xlabel('Number of epochs')
        plt.ylabel('Accuracy')

    plt.show()

Plot the validation accuracy vs the number of classes for CUB-200 positive SCOUTER with lambda value 10.

In [ ]:
# the number 0.628 was noted from a training output of the positive scouter file that got corrupted
accs = [0.628, pos_L10[3][50][-1], pos_L10[3][75][-1], pos_L10[3][100][-1]]

xs = [25, 50, 75, 100]


plt.plot(xs, accs)
plt.title('Validation accuracy as number of classes increases')
plt.xlabel('Number of classes')
plt.ylabel('Validation accuracy')

### Calculate metrics for ImageNet ResNest26 with 100 classes (area size, precision, IAUC, DAUC, sensitivity)

NOTE: this can take 1-2 hours per evaluation, in case of memory issues (like with DataLoader or other out of memory errors), 

please add/set argument --num_workers to 0 in the parameter list for the specific run, if that doesn't work you should run it on a more powerful GPU with more RAM (like on Google Cloud or Colab Pro).

In [ ]:
from get_results import *

# calculate metrics for positive scouter with lambda = 1
calculate_metrics(eval_Imagenet100_positive_lambda1)

In [ ]:
# calculate metrics for positive scouter with lambda = 3
calculate_metrics(eval_Imagenet100_positive_lambda3)

In [ ]:
# calculate metrics for positive scouter with lambda = 10
calculate_metrics(eval_Imagenet100_positive_lambda10)

In [ ]:
# calculate metrics for negative scouter with lambda = 1
calculate_metrics(eval_Imagenet100_negative_lambda1)

In [ ]:
# calculate metrics for negative scouter with lambda = 3
calculate_metrics(eval_Imagenet100_negative_lambda1)

In [ ]:
# calculate metrics for negative scouter with lambda = 10
calculate_metrics(eval_Imagenet100_negative_lambda1)